In [69]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import matplotlib as mpl
import anndata as ad
import matplotlib.patheffects as pe
import warnings
from scipy import sparse
import pickle

In [2]:
from tqdm import tqdm

In [70]:
new_rc_params = {'text.usetex': False,
"svg.fonttype": 'none'
}
mpl.rcParams.update(new_rc_params)
plt.rcParams.update({'axes.labelsize' : 16}) 

In [3]:
biological_types = ["1_W3L1",
                    "2_W3D1",
                    "3_F-mini-ON",
                    "4_F-mini-OFF",
                    "5_J-RGC",
                    "6_W3B",
                    "7_Novel",
                    "8_Novel",
                    "9_Tbr1-Novel",
                    "10_ooDSGC",
                    "11_Novel",
                    "12_N-ooDSGC",
                    "13_W3L2",
                    "14_Novel",
                    "15_Novel",
                    "16_D/V-ooDSGC",
                    "17_Tbr1-S1",
                    "18_Novel",
                    "19_Novel",
                    "20_Novel",
                    "21_Tbr1-S2",
                    "22_MX",
                    "23_W3D2",
                    "24_ooDSGC",
                    "25_Novel",
                    "26_Novel",
                    "27_Novel",
                    "28_F-midi-OFF",
                    "29_Novel",
                    "30_W3D3",
                    "31_M2",
                    "32_F-Novel",
                    "33_M1",
                    "34_Novel",
                    "35_Novel",
                    "36_Novel",
                    "37_Novel",
                    "38_F-midi-ON",
                    "39_Novel",
                    "40_M1-duplicate",
                    "41_alphaON-T",
                    "42_alphaOFF-S",
                    "43_alphaON-S/M4",
                    "44_Novel",
                    "45_alphaOFF-T"
]

In [4]:
p5_adata = sc.read_h5ad('path/to/adatas/P5_adata_typed.h5ad')
p7_adata = sc.read_h5ad('path/to/adatas/P7_adata_pruned.h5ad')
p56_adata = sc.read_h5ad('path/to/adatas/P56_adata.h5ad')

In [5]:
p7b2_adata = sc.read_h5ad('path/to/adatas/P7b2_adata_pruned.h5ad')

In [6]:
p56_adata.obs['P56_type'] = p56_adata.obs['Bio_type_cluster']

In [7]:
p7_p5_p56_combined_adata = ad.AnnData.concatenate(p7_adata, p7b2_adata, p5_adata, p56_adata, batch_key='Atlas', batch_categories=['P7','P7b2','P5','P56'])

C:\Users\pomat\AppData\Local\Programs\Python\Python311\Lib\site-packages\anndata\_core\anndata.py:1755: FutureWarning: The AnnData.concatenate method is deprecated in favour of the anndata.concat function. Please use anndata.concat instead.

See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  warnings.warn(


# Calculating Global LFCs

In [8]:
p5_adata_raw = p5_adata.raw
p7_adata_raw = p7_adata.raw
p56_adata_raw = p56_adata.raw

In [9]:
p7b2_adata_raw = p7b2_adata.raw

In [10]:
common_genes = []
for i in list(p5_adata_raw.var_names):
    if (i in p7_adata_raw.var_names) & (i in p56_adata_raw.var_names):
        common_genes.append(i)

In [11]:
p5_adata_slice = p5_adata_raw[:, common_genes]
p7_adata_slice = p7_adata_raw[:, common_genes]
p56_adata_slice = p56_adata_raw[:, common_genes]

In [12]:
p7b2_adata_slice = p7b2_adata_raw[:, common_genes]

In [13]:
p5_log_gem = p5_adata_slice.X
p7_log_gem = p7_adata_slice.X
p56_log_gem = p56_adata_slice.X

In [14]:
p7b2_log_gem = p7b2_adata_slice.X

In [15]:
p5_gem = p5_log_gem.expm1()
p7_gem = p7_log_gem.expm1()
p56_gem = p56_log_gem.expm1()

In [16]:
p7b2_gem = p7b2_log_gem.expm1()

In [17]:
p5_mean_exp = p5_gem.mean(0)
p7_mean_exp = p7_gem.mean(0)
p56_mean_exp = p56_gem.mean(0)

In [18]:
p7b2_mean_exp = p7b2_gem.mean(0)

In [19]:
p7_count = p7_adata_slice.X.shape[0]
p7b2_count = p7b2_adata_slice.X.shape[0]
p56_count = p56_adata_slice.X.shape[0]
p5_count = p5_adata_slice.X.shape[0]

p7_raw_X = p7_adata_slice.X
p7_raw_X.eliminate_zeros()
p7_raw_X_trans = p7_raw_X.transpose().tocsr()
p7_cell_counts = p7_raw_X_trans.indptr[1:] - p7_raw_X_trans.indptr[:-1]

p7b2_raw_X = p7b2_adata_slice.X
p7b2_raw_X.eliminate_zeros()
p7b2_raw_X_trans = p7b2_raw_X.transpose().tocsr()
p7b2_cell_counts = p7b2_raw_X_trans.indptr[1:] - p7b2_raw_X_trans.indptr[:-1]

p56_raw_X = p56_adata_slice.X
p56_raw_X.eliminate_zeros()
p56_raw_X_trans = p56_raw_X.transpose().tocsr()
p56_cell_counts = p56_raw_X_trans.indptr[1:] - p56_raw_X_trans.indptr[:-1]

p5_raw_X = p5_adata_slice.X
p5_raw_X.eliminate_zeros()
p5_raw_X_trans = p5_raw_X.transpose().tocsr()
p5_cell_counts = p5_raw_X_trans.indptr[1:] - p5_raw_X_trans.indptr[:-1]

p7_cell_pcts = p7_cell_counts/p7_count
p7b2_cell_pcts = p7b2_cell_counts/p7b2_count
p56_cell_pcts = p56_cell_counts/p56_count
p5_cell_pcts = p5_cell_counts/p5_count

In [20]:
gene_names = []
p7_p5_fold_changes = []
p7_p56_fold_changes = []
p56_p5_fold_changes = []
p56_p7_fold_changes = []
p7b2_p7_fold_changes = []
p7b2_p56_fold_changes = []
p7b2_p5_fold_changes = []
p7_p7b2_fold_changes = []
for i in range(p5_mean_exp.shape[1]):
    gene_name = list(p5_adata_slice.var_names)[i]
    
    p5_index = i
    p7_index = list(p7_adata_slice.var_names).index(gene_name)
    p56_index = list(p56_adata_slice.var_names).index(gene_name)
    
    gene_names.append(gene_name)
    p7_p5_fold_change = (p7_mean_exp[0, p7_index]+0.01)/(p5_mean_exp[0, p5_index]+0.01)
    p7_p5_fold_changes.append(p7_p5_fold_change)
    p7_p56_fold_change = (p7_mean_exp[0, p7_index]+0.01)/(p56_mean_exp[0, p56_index]+0.01)
    p7_p56_fold_changes.append(p7_p56_fold_change)
    p56_p7_fold_changes.append(1/p7_p56_fold_change)
    p56_p5_fold_change = (p56_mean_exp[0, p56_index]+0.01)/(p5_mean_exp[0, p5_index]+0.01)
    p56_p5_fold_changes.append(p56_p5_fold_change)
    p7b2_p7_fold_change = (p7b2_mean_exp[0, p7_index]+0.01)/(p7_mean_exp[0, p7_index]+0.01)
    p7b2_p7_fold_changes.append(p7b2_p7_fold_change)
    p7_p7b2_fold_changes.append(1/p7b2_p7_fold_change)
    p7b2_p56_fold_change = (p7b2_mean_exp[0, p7_index]+0.01)/(p56_mean_exp[0, p56_index]+0.01)
    p7b2_p56_fold_changes.append(p7b2_p56_fold_change)
    p7b2_p5_fold_change = (p7b2_mean_exp[0, p7_index]+0.01)/(p5_mean_exp[0, p5_index]+0.01)
    p7b2_p5_fold_changes.append(p7b2_p5_fold_change)

In [21]:
pickle.dump(p7_p5_fold_changes, open('p7_p5_fold_changes.pkl', 'wb'))
pickle.dump(p7_p56_fold_changes, open('p7_p56_fold_changes.pkl', 'wb'))
pickle.dump(p56_p5_fold_changes, open('p56_p5_fold_changes.pkl', 'wb'))
pickle.dump(p56_p7_fold_changes, open('p56_p7_fold_changes.pkl', 'wb'))
pickle.dump(p7b2_p7_fold_changes, open('p7b2_p7_fold_changes.pkl', 'wb'))
pickle.dump(p7b2_p56_fold_changes, open('p7b2_p56_fold_changes.pkl', 'wb'))
pickle.dump(p7b2_p5_fold_changes, open('p7b2_p5_fold_changes.pkl', 'wb'))
pickle.dump(p7_p7b2_fold_changes, open('p7_p7b2_fold_changes.pkl', 'wb'))

In [22]:
p7_p5_blacklist = []
p7_p56_blacklist = []
p7_p7b2_blacklist = []
p5_p56_blacklist = []
p5_p7b2_blacklist = []
p56_p7b2_blacklist = []
for i in range(p5_mean_exp.shape[1]):
    gene_name = list(p5_adata_slice.var_names)[i]
    
    p5_index = i
    p7_index = list(p7_adata_slice.var_names).index(gene_name)
    p56_index = list(p56_adata_slice.var_names).index(gene_name)

    if p7_cell_pcts[p7_index] < 0.05:
        if p5_cell_pcts[p5_index] < 0.05:
            p7_p5_blacklist.append(i)
        if p56_cell_pcts[p56_index] < 0.05:
            p7_p56_blacklist.append(i)
        if p7b2_cell_pcts[p7_index] < 0.05:
            p7_p7b2_blacklist.append(i)
    if p5_cell_pcts[p5_index] < 0.05:
        if p56_cell_pcts[p56_index] < 0.05:
            p5_p56_blacklist.append(i)
        if p7b2_cell_pcts[p7_index] < 0.05:
            p5_p7b2_blacklist.append(i)
    if p56_cell_pcts[p56_index] < 0.05:
        if p7b2_cell_pcts[p7_index] < 0.05:
            p56_p7b2_blacklist.append(i)

In [23]:
pickle.dump(p7_p5_blacklist, open('p7_p5_blacklist.pkl', 'wb'))
pickle.dump(p7_p56_blacklist, open('p7_p56_blacklist.pkl', 'wb'))
pickle.dump(p7_p7b2_blacklist, open('p7_p7b2_blacklist.pkl', 'wb'))
pickle.dump(p5_p56_blacklist, open('p5_p56_blacklist.pkl', 'wb'))
pickle.dump(p5_p7b2_blacklist, open('p5_p7b2_blacklist.pkl', 'wb'))
pickle.dump(p56_p7b2_blacklist, open('p56_p7b2_blacklist.pkl', 'wb'))

# Calculating Inter-Replicate LFCs

In [31]:
p7rep1_adata_raw = p7_adata[p7_adata.obs["Batch"] == "Batch1"].raw
p7rep2_adata_raw = p7_adata[p7_adata.obs["Batch"] == "Batch2"].raw
p7rep3_adata_raw = p7_adata[p7_adata.obs["Batch"] == "Batch3"].raw

In [32]:
p7b2rep1_adata_raw = p7b2_adata[p7b2_adata.obs["Batch"].isin(["Batch1", "Batch2"])].raw
p7b2rep2_adata_raw = p7b2_adata[p7b2_adata.obs["Batch"] == "Batch1_2"].raw
p7b2rep3_adata_raw = p7b2_adata[p7b2_adata.obs["Batch"] == "Batch2_2"].raw

In [33]:
p7rep1_adata_slice = p7rep1_adata_raw[:, common_genes]
p7rep2_adata_slice = p7rep2_adata_raw[:, common_genes]
p7rep3_adata_slice = p7rep3_adata_raw[:, common_genes]

In [34]:
p7b2rep1_adata_slice = p7b2rep1_adata_raw[:, common_genes]
p7b2rep2_adata_slice = p7b2rep2_adata_raw[:, common_genes]
p7b2rep3_adata_slice = p7b2rep3_adata_raw[:, common_genes]

In [35]:
p7rep1_log_gem = p7rep1_adata_slice.X
p7rep2_log_gem = p7rep2_adata_slice.X
p7rep3_log_gem = p7rep3_adata_slice.X

In [36]:
p7b2rep1_log_gem = p7b2rep1_adata_slice.X
p7b2rep2_log_gem = p7b2rep2_adata_slice.X
p7b2rep3_log_gem = p7b2rep3_adata_slice.X

In [37]:
p7rep1_gem = p7rep1_log_gem.expm1()
p7rep2_gem = p7rep2_log_gem.expm1()
p7rep3_gem = p7rep3_log_gem.expm1()

In [38]:
p7b2rep1_gem = p7b2rep1_log_gem.expm1()
p7b2rep2_gem = p7b2rep2_log_gem.expm1()
p7b2rep3_gem = p7b2rep3_log_gem.expm1()

In [39]:
p7rep1_mean_exp = p7rep1_gem.mean(0)
p7rep2_mean_exp = p7rep2_gem.mean(0)
p7rep3_mean_exp = p7rep3_gem.mean(0)

In [40]:
p7b2rep1_mean_exp = p7b2rep1_gem.mean(0)
p7b2rep2_mean_exp = p7b2rep2_gem.mean(0)
p7b2rep3_mean_exp = p7b2rep3_gem.mean(0)

In [41]:
p7rep1_count = p7rep1_adata_slice.X.shape[0]
p7rep2_count = p7rep2_adata_slice.X.shape[0]
p7rep3_count = p7rep3_adata_slice.X.shape[0]
p7b2rep1_count = p7b2rep1_adata_slice.X.shape[0]
p7b2rep2_count = p7b2rep2_adata_slice.X.shape[0]
p7b2rep3_count = p7b2rep3_adata_slice.X.shape[0]

p7rep1_raw_X = p7rep1_adata_slice.X
p7rep1_raw_X.eliminate_zeros()
p7rep1_raw_X_trans = p7rep1_raw_X.transpose().tocsr()
p7rep1_cell_counts = p7rep1_raw_X_trans.indptr[1:] - p7rep1_raw_X_trans.indptr[:-1]

p7rep2_raw_X = p7rep2_adata_slice.X
p7rep2_raw_X.eliminate_zeros()
p7rep2_raw_X_trans = p7rep2_raw_X.transpose().tocsr()
p7rep2_cell_counts = p7rep2_raw_X_trans.indptr[1:] - p7rep2_raw_X_trans.indptr[:-1]

p7rep3_raw_X = p7rep3_adata_slice.X
p7rep3_raw_X.eliminate_zeros()
p7rep3_raw_X_trans = p7rep3_raw_X.transpose().tocsr()
p7rep3_cell_counts = p7rep3_raw_X_trans.indptr[1:] - p7rep3_raw_X_trans.indptr[:-1]

p7b2rep1_raw_X = p7b2rep1_adata_slice.X
p7b2rep1_raw_X.eliminate_zeros()
p7b2rep1_raw_X_trans = p7b2rep1_raw_X.transpose().tocsr()
p7b2rep1_cell_counts = p7b2rep1_raw_X_trans.indptr[1:] - p7b2rep1_raw_X_trans.indptr[:-1]

p7b2rep2_raw_X = p7b2rep2_adata_slice.X
p7b2rep2_raw_X.eliminate_zeros()
p7b2rep2_raw_X_trans = p7b2rep2_raw_X.transpose().tocsr()
p7b2rep2_cell_counts = p7b2rep2_raw_X_trans.indptr[1:] - p7b2rep2_raw_X_trans.indptr[:-1]

p7b2rep3_raw_X = p7b2rep3_adata_slice.X
p7b2rep3_raw_X.eliminate_zeros()
p7b2rep3_raw_X_trans = p7b2rep3_raw_X.transpose().tocsr()
p7b2rep3_cell_counts = p7b2rep3_raw_X_trans.indptr[1:] - p7b2rep3_raw_X_trans.indptr[:-1]

p7rep1_cell_pcts = p7rep1_cell_counts/p7rep1_count
p7rep2_cell_pcts = p7rep2_cell_counts/p7rep2_count
p7rep3_cell_pcts = p7rep3_cell_counts/p7rep3_count
p7b2rep1_cell_pcts = p7b2rep1_cell_counts/p7b2rep1_count
p7b2rep2_cell_pcts = p7b2rep2_cell_counts/p7b2rep2_count
p7b2rep3_cell_pcts = p7b2rep3_cell_counts/p7b2rep3_count

In [42]:
gene_names = []
p7rep1_p7rep2_fold_changes = []
p7rep1_p7rep3_fold_changes = []
p7rep2_p7rep3_fold_changes = []
p7b2rep1_p7b2rep2_fold_changes = []
p7b2rep1_p7b2rep3_fold_changes = []
p7b2rep2_p7b2rep3_fold_changes = []
for i in range(p5_mean_exp.shape[1]):
    gene_name = list(p5_adata_slice.var_names)[i]
    p7_index = list(p7_adata_slice.var_names).index(gene_name)
    
    #if (p5_mean_exp[0, p5_index] != 0) & (p7_mean_exp[0, p7_index] != 0) & (p56_mean_exp[0, p56_index] != 0) & (p7Batch2_mean_exp[0, p7_index] != 0) & (p7Batch3_mean_exp[0, p7_index] != 0) & (p5Batch3_mean_exp[0, p5_index] != 0) & (p5Batch5_mean_exp[0, p5_index] != 0):
    gene_names.append(gene_name)
    p7rep1_p7rep2_fold_change = (p7rep1_mean_exp[0, p7_index]+0.01)/(p7rep2_mean_exp[0, p7_index]+0.01)
    p7rep1_p7rep2_fold_changes.append(p7rep1_p7rep2_fold_change)
    p7rep1_p7rep3_fold_change = (p7rep1_mean_exp[0, p7_index]+0.01)/(p7rep3_mean_exp[0, p7_index]+0.01)
    p7rep1_p7rep3_fold_changes.append(p7rep1_p7rep3_fold_change)
    p7rep2_p7rep3_fold_change = (p7rep2_mean_exp[0, p7_index]+0.01)/(p7rep3_mean_exp[0, p7_index]+0.01)
    p7rep2_p7rep3_fold_changes.append(p7rep2_p7rep3_fold_change)

    p7b2rep1_p7b2rep2_fold_change = (p7b2rep1_mean_exp[0, p7_index]+0.01)/(p7b2rep2_mean_exp[0, p7_index]+0.01)
    p7b2rep1_p7b2rep2_fold_changes.append(p7b2rep1_p7b2rep2_fold_change)
    p7b2rep1_p7b2rep3_fold_change = (p7b2rep1_mean_exp[0, p7_index]+0.01)/(p7b2rep3_mean_exp[0, p7_index]+0.01)
    p7b2rep1_p7b2rep3_fold_changes.append(p7b2rep1_p7b2rep3_fold_change)
    p7b2rep2_p7b2rep3_fold_change = (p7b2rep2_mean_exp[0, p7_index]+0.01)/(p7b2rep3_mean_exp[0, p7_index]+0.01)
    p7b2rep2_p7b2rep3_fold_changes.append(p7b2rep2_p7b2rep3_fold_change)

In [43]:
pickle.dump(p7rep1_p7rep2_fold_changes, open('p7rep1_p7rep2_fold_changes.pkl', 'wb'))
pickle.dump(p7rep1_p7rep3_fold_changes, open('p7rep1_p7rep3_fold_changes.pkl', 'wb'))
pickle.dump(p7rep2_p7rep3_fold_changes, open('p7rep2_p7rep3_fold_changes.pkl', 'wb'))
pickle.dump(p7b2rep1_p7b2rep2_fold_changes, open('p7b2rep1_p7b2rep2_fold_changes.pkl', 'wb'))
pickle.dump(p7b2rep1_p7b2rep3_fold_changes, open('p7b2rep1_p7b2rep3_fold_changes.pkl', 'wb'))
pickle.dump(p7b2rep2_p7b2rep3_fold_changes, open('p7b2rep2_p7b2rep3_fold_changes.pkl', 'wb'))

In [44]:
p7rep1_p7rep2_blacklist = []
p7rep1_p7rep3_blacklist = []
p7rep2_p7rep3_blacklist = []
p7b2rep1_p7b2rep2_blacklist = []
p7b2rep1_p7b2rep3_blacklist = []
p7b2rep2_p7b2rep3_blacklist = []
for i in range(p5_mean_exp.shape[1]):
    gene_name = list(p5_adata_slice.var_names)[i]
    p7_index = list(p7_adata_slice.var_names).index(gene_name)

    if p7rep1_cell_pcts[p7_index] < 0.05:
        if p7rep2_cell_pcts[p7_index] < 0.05:
            p7rep1_p7rep2_blacklist.append(i)
        if p7rep3_cell_pcts[p7_index] < 0.05:
            p7rep1_p7rep3_blacklist.append(i)
    if p7rep2_cell_pcts[p7_index] < 0.05:
        if p7rep3_cell_pcts[p7_index] < 0.05:
            p7rep2_p7rep3_blacklist.append(i)
    if p7b2rep1_cell_pcts[p7_index] < 0.05:
        if p7b2rep2_cell_pcts[p7_index] < 0.05:
            p7b2rep1_p7b2rep2_blacklist.append(i)
        if p7b2rep3_cell_pcts[p7_index] < 0.05:
            p7b2rep1_p7b2rep3_blacklist.append(i)
    if p7b2rep2_cell_pcts[p7_index] < 0.05:
        if p7b2rep3_cell_pcts[p7_index] < 0.05:
            p7b2rep2_p7b2rep3_blacklist.append(i)

In [45]:
pickle.dump(p7rep1_p7rep2_blacklist, open('p7rep1_p7rep2_blacklist.pkl', 'wb'))
pickle.dump(p7rep1_p7rep3_blacklist, open('p7rep1_p7rep3_blacklist.pkl', 'wb'))
pickle.dump(p7rep2_p7rep3_blacklist, open('p7rep2_p7rep3_blacklist.pkl', 'wb'))
pickle.dump(p7b2rep1_p7b2rep2_blacklist, open('p7b2rep1_p7b2rep2_blacklist.pkl', 'wb'))
pickle.dump(p7b2rep1_p7b2rep3_blacklist, open('p7b2rep1_p7b2rep3_blacklist.pkl', 'wb'))
pickle.dump(p7b2rep2_p7b2rep3_blacklist, open('p7b2rep2_p7b2rep3_blacklist.pkl', 'wb'))

# By Type

In [24]:
p56_p56_types = []
for i in p56_adata.obs['P56_type']:
    p56_p56_types.append(int(i))
p56_adata.obs['P56_type'] = pd.Categorical(p56_p56_types)

In [25]:
p5_mean_exp_by_type = []
p7_mean_exp_by_type = []
p56_mean_exp_by_type = []
p7b2_mean_exp_by_type = []

p5_cell_pcts_by_type = []
p7_cell_pcts_by_type = []
p56_cell_pcts_by_type = []
p7b2_cell_pcts_by_type = []
for i in biological_types:
    P56_type = int(i.split('_')[0])
    
    p5_adata_raw_type = p5_adata[p5_adata.obs['P56_type'] == P56_type].raw
    p7_adata_raw_type = p7_adata[p7_adata.obs['P56_type'] == P56_type].raw
    p56_adata_raw_type = p56_adata[p56_adata.obs['P56_type'] == P56_type].raw
    p7b2_adata_raw_type = p7b2_adata[p7b2_adata.obs['P56_type'] == P56_type].raw
    
    p5_adata_slice_type = p5_adata_raw_type[:, common_genes]
    p7_adata_slice_type = p7_adata_raw_type[:, common_genes]
    p56_adata_slice_type = p56_adata_raw_type[:, common_genes]
    p7b2_adata_slice_type = p7b2_adata_raw_type[:, common_genes]
    
    p5_log_gem_type = p5_adata_slice_type.X
    p7_log_gem_type = p7_adata_slice_type.X
    p56_log_gem_type = p56_adata_slice_type.X
    p7b2_log_gem_type = p7b2_adata_slice_type.X
    
    p5_gem_type = p5_log_gem_type.expm1()
    p7_gem_type = p7_log_gem_type.expm1()
    p56_gem_type = p56_log_gem_type.expm1()
    p7b2_gem_type = p7b2_log_gem_type.expm1()
    
    p5_mean_exp_type = p5_gem_type.mean(0)
    p7_mean_exp_type = p7_gem_type.mean(0)
    p56_mean_exp_type = p56_gem_type.mean(0)
    p7b2_mean_exp_type = p7b2_gem_type.mean(0)
    
    p5_mean_exp_by_type.append(p5_mean_exp_type)
    p7_mean_exp_by_type.append(p7_mean_exp_type)
    p56_mean_exp_by_type.append(p56_mean_exp_type)
    p7b2_mean_exp_by_type.append(p7b2_mean_exp_type)
    
    p7_count_type = p7_adata_slice_type.X.shape[0]
    p7b2_count_type = p7b2_adata_slice_type.X.shape[0]
    p56_count_type = p56_adata_slice_type.X.shape[0]
    p5_count_type = p5_adata_slice_type.X.shape[0]
    
    p7_raw_X_type = p7_adata_slice_type.X
    p7_raw_X_type.eliminate_zeros()
    p7_raw_X_trans_type = p7_raw_X_type.transpose().tocsr()
    p7_cell_counts_type = p7_raw_X_trans_type.indptr[1:] - p7_raw_X_trans_type.indptr[:-1]
    
    p7b2_raw_X_type = p7b2_adata_slice_type.X
    p7b2_raw_X_type.eliminate_zeros()
    p7b2_raw_X_trans_type = p7b2_raw_X_type.transpose().tocsr()
    p7b2_cell_counts_type = p7b2_raw_X_trans_type.indptr[1:] - p7b2_raw_X_trans_type.indptr[:-1]
    
    p56_raw_X_type = p56_adata_slice_type.X
    p56_raw_X_type.eliminate_zeros()
    p56_raw_X_trans_type = p56_raw_X_type.transpose().tocsr()
    p56_cell_counts_type = p56_raw_X_trans_type.indptr[1:] - p56_raw_X_trans_type.indptr[:-1]
    
    p5_raw_X_type = p5_adata_slice_type.X
    p5_raw_X_type.eliminate_zeros()
    p5_raw_X_trans_type = p5_raw_X_type.transpose().tocsr()
    p5_cell_counts_type = p5_raw_X_trans_type.indptr[1:] - p5_raw_X_trans_type.indptr[:-1]
    
    p7_cell_pcts_type = p7_cell_counts_type/p7_count_type
    p7b2_cell_pcts_type = p7b2_cell_counts_type/p7b2_count_type
    p56_cell_pcts_type = p56_cell_counts_type/p56_count_type
    p5_cell_pcts_type = p5_cell_counts_type/p5_count_type

    p7_cell_pcts_by_type.append(p7_cell_pcts_type)
    p7b2_cell_pcts_by_type.append(p7b2_cell_pcts_type)
    p56_cell_pcts_by_type.append(p56_cell_pcts_type)
    p5_cell_pcts_by_type.append(p5_cell_pcts_type)

    print(P56_type)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45


In [26]:
p7_p5_fold_changes_by_type = []
p7_p56_fold_changes_by_type = []
p56_p5_fold_changes_by_type = []
p56_p7_fold_changes_by_type = []
p7b2_p7_fold_changes_by_type = []
p7b2_p56_fold_changes_by_type = []
p7b2_p5_fold_changes_by_type = []
p7_p7b2_fold_changes_by_type = []
for j in tqdm(range(len(biological_types))):
    p7_p5_fold_changes_type = []
    p7_p56_fold_changes_type = []
    p56_p5_fold_changes_type = []
    p56_p7_fold_changes_type = []
    p7b2_p7_fold_changes_type = []
    p7b2_p56_fold_changes_type = []
    p7b2_p5_fold_changes_type = []
    p7_p7b2_fold_changes_type = []
    for i in range(p5_mean_exp.shape[1]):
        gene_name = list(p5_adata_slice.var_names)[i]
        
        p5_index = i
        p7_index = list(p7_adata_slice.var_names).index(gene_name)
        p56_index = list(p56_adata_slice.var_names).index(gene_name)

        pseudocount = 0.1
        #if (p5_mean_exp[0, p5_index] != 0) & (p7_mean_exp[0, p7_index] != 0) & (p56_mean_exp[0, p56_index] != 0) & (p7Batch2_mean_exp[0, p7_index] != 0) & (p7Batch3_mean_exp[0, p7_index] != 0) & (p5Batch3_mean_exp[0, p5_index] != 0) & (p5Batch5_mean_exp[0, p5_index] != 0):
        # gene_names.append(gene_name)
        p7_p5_fold_change = (p7_mean_exp_by_type[j][0, p7_index]+pseudocount)/(p5_mean_exp_by_type[j][0, p5_index]+pseudocount)
        p7_p5_fold_changes_type.append(p7_p5_fold_change)
        p7_p56_fold_change = (p7_mean_exp_by_type[j][0, p7_index]+pseudocount)/(p56_mean_exp_by_type[j][0, p56_index]+pseudocount)
        p7_p56_fold_changes_type.append(p7_p56_fold_change)
        p56_p7_fold_changes_type.append(1/p7_p56_fold_change)
        p56_p5_fold_change = (p56_mean_exp_by_type[j][0, p56_index]+pseudocount)/(p5_mean_exp_by_type[j][0, p5_index]+pseudocount)
        p56_p5_fold_changes_type.append(p56_p5_fold_change)
        p7b2_p7_fold_change = (p7b2_mean_exp_by_type[j][0, p7_index]+pseudocount)/(p7_mean_exp_by_type[j][0, p7_index]+pseudocount)
        p7b2_p7_fold_changes_type.append(p7b2_p7_fold_change)
        p7_p7b2_fold_changes_type.append(1/p7b2_p7_fold_change)
        p7b2_p56_fold_change = (p7b2_mean_exp_by_type[j][0, p7_index]+pseudocount)/(p56_mean_exp_by_type[j][0, p56_index]+pseudocount)
        p7b2_p56_fold_changes_type.append(p7b2_p56_fold_change)
        p7b2_p5_fold_change = (p7b2_mean_exp_by_type[j][0, p7_index]+pseudocount)/(p5_mean_exp_by_type[j][0, p5_index]+pseudocount)
        p7b2_p5_fold_changes_type.append(p7b2_p5_fold_change)
    p7_p5_fold_changes_by_type.append(p7_p5_fold_changes_type)
    p7_p56_fold_changes_by_type.append(p7_p56_fold_changes_type)
    p56_p5_fold_changes_by_type.append(p56_p5_fold_changes_type)
    p56_p7_fold_changes_by_type.append(p56_p7_fold_changes_type)
    p7b2_p7_fold_changes_by_type.append(p7b2_p7_fold_changes_type)
    p7b2_p56_fold_changes_by_type.append(p7b2_p56_fold_changes_type)
    p7b2_p5_fold_changes_by_type.append(p7b2_p5_fold_changes_type)
    p7_p7b2_fold_changes_by_type.append(p7_p7b2_fold_changes_type)

100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [20:48<00:00, 27.74s/it]


In [27]:
import pickle

In [46]:
pickle.dump(p7_p5_fold_changes_by_type, open('p7_p5_fold_changes_by_type.pkl', 'wb'))
pickle.dump(p7_p56_fold_changes_by_type, open('p7_p56_fold_changes_by_type.pkl', 'wb'))
pickle.dump(p56_p5_fold_changes_by_type, open('p56_p5_fold_changes_by_type.pkl', 'wb'))
pickle.dump(p56_p7_fold_changes_by_type, open('p56_p7_fold_changes_by_type.pkl', 'wb'))
pickle.dump(p7b2_p7_fold_changes_by_type, open('p7b2_p7_fold_changes_by_type.pkl', 'wb'))
pickle.dump(p7b2_p56_fold_changes_by_type, open('p7b2_p56_fold_changes_by_type.pkl', 'wb'))
pickle.dump(p7b2_p5_fold_changes_by_type, open('p7b2_p5_fold_changes_by_type.pkl', 'wb'))
pickle.dump(p7_p7b2_fold_changes_by_type, open('p7_p7b2_fold_changes_by_type.pkl', 'wb'))

In [29]:
p7_p5_blacklist_by_type = []
p7_p56_blacklist_by_type = []
p7_p7b2_blacklist_by_type = []
p5_p56_blacklist_by_type = []
p5_p7b2_blacklist_by_type = []
p56_p7b2_blacklist_by_type = []
for j in tqdm(range(len(biological_types))):
    p7_num = len(p7_adata[p7_adata.obs['P56_type'] == j].obs)
    p7b2_num = len(p7b2_adata[p7b2_adata.obs['P56_type'] == j].obs)
    p5_num = len(p5_adata[p5_adata.obs['P56_type'] == j].obs)
    p56_num = len(p56_adata[p56_adata.obs['P56_type'] == j].obs)
    # if (p7_num < 40) or (p7b2_num < 40) or (p5_num < 40) or (p56_num < 40):
    #     worldwide_percentage = 0.2
    # else:
    #     worldwide_percentage = 0.05
    p7_p5_blacklist_type = []
    p7_p56_blacklist_type = []
    p7_p7b2_blacklist_type = []
    p5_p56_blacklist_type = []
    p5_p7b2_blacklist_type = []
    p56_p7b2_blacklist_type = []
    for i in range(p5_mean_exp.shape[1]):
        gene_name = list(p5_adata_slice.var_names)[i]
        
        p5_index = i
        p7_index = list(p7_adata_slice.var_names).index(gene_name)
        p56_index = list(p56_adata_slice.var_names).index(gene_name)
    
        if p7_cell_pcts_by_type[j][p7_index] < 0.05:
            if p5_cell_pcts_by_type[j][p5_index] < 0.05:
                p7_p5_blacklist_type.append(i)
            if p56_cell_pcts_by_type[j][p56_index] < 0.05:
                p7_p56_blacklist_type.append(i)
            if p7b2_cell_pcts_by_type[j][p7_index] < 0.05:
                p7_p7b2_blacklist_type.append(i)
        if p5_cell_pcts_by_type[j][p5_index] < 0.05:
            if p56_cell_pcts_by_type[j][p56_index] < 0.05:
                p5_p56_blacklist_type.append(i)
            if p7b2_cell_pcts_by_type[j][p7_index] < 0.05:
                p5_p7b2_blacklist_type.append(i)
        if p56_cell_pcts_by_type[j][p56_index] < 0.05:
            if p7b2_cell_pcts_by_type[j][p7_index] < 0.05:
                p56_p7b2_blacklist_type.append(i)
    p7_p5_blacklist_by_type.append(p7_p5_blacklist_type)
    p7_p56_blacklist_by_type.append(p7_p5_blacklist_type)
    p7_p7b2_blacklist_by_type.append(p7_p5_blacklist_type)
    p5_p56_blacklist_by_type.append(p7_p5_blacklist_type)
    p5_p7b2_blacklist_by_type.append(p7_p5_blacklist_type)
    p56_p7b2_blacklist_by_type.append(p7_p5_blacklist_type)

100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [20:39<00:00, 27.55s/it]


In [30]:
pickle.dump(p7_p5_blacklist_by_type, open('p7_p5_blacklist_by_type.pkl', 'wb'))
pickle.dump(p7_p56_blacklist_by_type, open('p7_p56_blacklist_by_type.pkl', 'wb'))
pickle.dump(p7_p7b2_blacklist_by_type, open('p7_p7b2_blacklist_by_type.pkl', 'wb'))
pickle.dump(p5_p56_blacklist_by_type, open('p5_p56_blacklist_by_type.pkl', 'wb'))
pickle.dump(p5_p7b2_blacklist_by_type, open('p5_p7b2_blacklist_by_type.pkl', 'wb'))
pickle.dump(p56_p7b2_blacklist_by_type, open('p56_p7b2_blacklist_by_type.pkl', 'wb'))